# This is Work in Progress (incomplete & undocumented)

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Read NaturalEarth data of airpoirts of the world
# This data is available as shapefile, but for demonstration purposes
# a csv was created that is read in here.
df_airports = pd.read_csv('./inData/ne_10m_airports/ne_10m_airports.csv')
airports = gpd.GeoDataFrame(
    df_airports,
    crs='EPSG:4326',
    geometry=gpd.points_from_xy(df_airports.longitude, df_airports.latitude))
del df_airports
# Create spatial index
airports.sindex

print(airports.shape)
airports.head(2)

In [ ]:
airports.explore()

In [ ]:
countries = gpd.read_file('./inData/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp')
countries = countries[['NAME', 'SUBREGION', 'REGION_UN', 'geometry']]
countries.head(2)

In [ ]:
countries.explore()

In [ ]:
countries_subregion = countries\
    .drop(columns=['NAME'])\
    .dissolve(by='SUBREGION')\
    .reset_index()
countries_subregion.sindex
countries_subregion.head(5)

In [ ]:
countries_subregion.explore()

In [ ]:
regions_with_airports = gpd.sjoin(countries_subregion, airports, how='inner', predicate='contains')
regions_with_airports.head(3)

In [ ]:
count_per_region = regions_with_airports\
    .groupby('SUBREGION')\
    .agg({
        'geometry': 'first',
        'index_right': 'count'
    })\
    .reset_index()\
    .rename(columns={
        'index_right': 'airport_count'
    })
count_per_region.head(3)

In [ ]:
count_per_region.plot(
    figsize=(18,6),
    column='airport_count', 
    cmap='YlOrRd',
    legend=True)